In [ ]:
import wandb

# wandb 로그인
wandb.login()

c:\Users\didsu\miniconda3\envs\link_dl\lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
c:\Users\didsu\miniconda3\envs\link_dl\lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annot

True

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import os

from pathlib import Path
import os
BASE_PATH = os.getcwd()  
print("BASE_PATH:", BASE_PATH)

import sys
sys.path.append(BASE_PATH)

from titanic_dataset import get_preprocessed_dataset


def get_data():
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  print(len(train_dataset), len(validation_dataset), len(test_dataset))

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
  test_data_loader = DataLoader(dataset= test_dataset, batch_size=len(test_dataset))
  return train_data_loader, validation_data_loader, test_data_loader


class MyModel(nn.Module):
  def __init__(self, n_input, n_output, activation_fn): # 이 줄을 수정
    super().__init__()
    # 문자열 이름과 실제 활성화 함수 클래스를 매핑
    activation_functions = { # 이 줄을 추가
        'ReLU': nn.ReLU(),
        'Sigmoid': nn.Sigmoid(),
        'ELU': nn.ELU(),
        'Leaky ReLU': nn.LeakyReLU()
    }

# BCEWithLogitsLoss용
    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      activation_functions[activation_fn], # 이 줄을 수정
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      activation_functions[activation_fn], # 이 줄을 수정
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x


def get_model_and_optimizer():
  my_model = MyModel(n_input=10, n_output=1, activation_fn=wandb.config.activation_fn) # 이 줄 수정
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer


def training_loop(model, optimizer, train_data_loader, validation_data_loader, test_data_loader): 
  n_epochs = wandb.config.epochs
  loss_fn = nn.BCEWithLogitsLoss()
  next_print_epoch = 100

  # Early stopping 변수 추가(하단의 validation loss까지 구해진 다음에 early stopping을 할지를 결정하는 부분이라 많은 코드가 요구되지 않음)
  best_validation_loss = float('inf')
  patience_counter = 0
  best_model_state = None

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input = train_batch['input']
      target = train_batch['target'].float().unsqueeze(1)  # Float 변환 & [512] → [512, 1]
      output_train = model(input)
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()


    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        # input, target = validation_batch
        input = validation_batch['input']
        output_validation = model(input)
        target = validation_batch['target'].float().unsqueeze(1)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    avg_validation_loss = loss_validation / num_validations

    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": avg_validation_loss # loss_validation / num_validations
    })
      # Early stopping 체크
    if avg_validation_loss < best_validation_loss:
      best_validation_loss = avg_validation_loss
      patience_counter = 0
      best_model_state = model.state_dict().copy()  # 최고 모델 저장
    else:
      patience_counter += 1

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
        f"Patience: {patience_counter}/{wandb.config.patience}"
      )
      next_print_epoch += 100

      # Early stopping 조건
    if patience_counter >= wandb.config.patience:
      print(f"\n Early stopping at epoch {epoch}")
      print(f"Best validation loss: {best_validation_loss:.4f}")
      model.load_state_dict(best_model_state)  # 최고 모델 복원
      break

def predict_test(model, test_data_loader):
  """Test 데이터로 예측 수행"""
  model.eval()
  predictions = []
  
  with torch.no_grad():
    for test_batch in test_data_loader:
      input = test_batch['input']
      output = model(input)
      predictions.append(output)
  
  predictions = torch.cat(predictions, dim=0).squeeze()  # [N, 1] → [N]
  return predictions


def save_predictions_to_csv(predictions, filename="submission.csv"):
  """예측 결과를 CSV로 저장 (Kaggle 제출 형식)"""
  import pandas as pd
  
  # Kaggle test.csv의 PassengerId 읽기
  CURRENT_FILE_PATH = os.getcwd()
  test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
  test_df = pd.read_csv(test_data_path)
  
  # 예측값을 0 또는 1로 변환 (이진 분류)
  predictions_binary = (torch.sigmoid(predictions) > 0.5).long().numpy()
  
  # 제출 파일 생성
  submission_df = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': predictions_binary
  })
  
  submission_df.to_csv(filename, index=False)
  print(f"✅ Predictions saved to {filename}")
  return submission_df

def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
    'activation_fn': args.activation_fn,
    'patience': args.patience,
  }

  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="my_model_training",
    notes="My first wandb experiment",
    tags=["my_model", "Titanic"],
    # name=current_time_str,
    name = args.name,
    config=config
  )
  print(args)
  print(wandb.config)

  train_data_loader, validation_data_loader, test_data_loader = get_data()

  linear_model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader,
    test_data_loader=test_data_loader
  )
  # Test 예측 및 CSV 저장
  print("\n" + "#" * 50)
  print("Generating predictions for test data...")
  test_predictions = predict_test(linear_model, test_data_loader)
  
  # 파일명에 run name 포함 (그래서 wandb에서 보이는 이름이 file name으로 저장되게 하는 코드임) -> ex) name: ReLU -> ReLU_submission.csv
  # filename = f"submission_{args.name if args.name else current_time_str}.csv"
  # 하단 코드는 submission으로만 만들어지게 함
  save_predictions_to_csv(test_predictions)

  wandb.finish()


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    class Args:
        def __init__(self):
            self.wandb = True  # wandb 사용 여부
            self.batch_size = 128
            self.epochs = 5000
            self.name = "ReLU_b128_early"
            self.activation_fn = "ReLU"
            self.patience = 100

    args = Args()
    main(args)



BASE_PATH: c:\Users\didsu\workspace\deeplearning\link_dl\_03_homeworks\homework_2


{'epochs': 5000, 'batch_size': 128, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20], 'activation_fn': 'ReLU', 'patience': 100}
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333    

c:\Users\didsu\workspace\deeplearning\link_dl\_03_homeworks\homework_2\titanic_dataset.py:129: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
c:\Users\didsu\workspace\deeplearning\link_dl\_03_homeworks\homework_2\titanic_dataset.py:148: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

Epoch 100, Training loss 0.5960, Validation loss 0.6630Patience: 9/100
Epoch 200, Training loss 0.5901, Validation loss 0.6588Patience: 9/100
Epoch 300, Training loss 0.5873, Validation loss 0.6552Patience: 4/100
Epoch 400, Training loss 0.5800, Validation loss 0.6532Patience: 22/100
Epoch 500, Training loss 0.5841, Validation loss 0.6497Patience: 4/100
Epoch 600, Training loss 0.5823, Validation loss 0.6483Patience: 14/100
Epoch 700, Training loss 0.5794, Validation loss 0.6479Patience: 36/100
Epoch 800, Training loss 0.5713, Validation loss 0.6470Patience: 12/100
Epoch 900, Training loss 0.5740, Validation loss 0.6441Patience: 7/100
Epoch 1000, Training loss 0.5748, Validation loss 0.6418Patience: 2/100
Epoch 1100, Training loss 0.5691, Validation loss 0.6405Patience: 6/100
Epoch 1200, Training loss 0.5690, Validation loss 0.6390Patience: 1/100
Epoch 1300, Training loss 0.5655, Validation loss 0.6387Patience: 17/100
Epoch 1400, Training loss 0.5653, Validation loss 0.6376Patience: 33

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 5000, Training loss 0.4565, Validation loss 0.5090Patience: 21/100

##################################################
Generating predictions for test data...
✅ Predictions saved to submission.csv


Epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇██
Training loss,██▇██▇▇▇▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▂▂▁▁▁
Validation loss,█████▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▂▁
Epoch,5000
Training loss,0.45649
Validation loss,0.50897


# 각 셀별로 실행하기

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import os
import sys
import pandas as pd

# 경로 설정
BASE_PATH = os.getcwd()
print("BASE_PATH:", BASE_PATH)
sys.path.append(BASE_PATH)

from titanic_dataset import get_preprocessed_dataset

# %%
# 데이터 로드 함수
def get_data(batch_size):
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
    print(f"Train: {len(train_dataset)}, Validation: {len(validation_dataset)}, Test: {len(test_dataset)}")
    
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))
    
    return train_data_loader, validation_data_loader, test_data_loader

# %%
# 모델 정의
class MyModel(nn.Module):
    def __init__(self, n_input, n_output, n_hidden_units, activation_fn):
        super().__init__()
        # 활성화 함수 매핑
        activation_functions = {
            'ReLU': nn.ReLU(),
            'Sigmoid': nn.Sigmoid(),
            'ELU': nn.ELU(),
            'Leaky ReLU': nn.LeakyReLU()
        }
        
        # BCEWithLogitsLoss용 모델 (마지막에 활성화 함수 없음)
        self.model = nn.Sequential(
            nn.Linear(n_input, n_hidden_units[0]),
            activation_functions[activation_fn],
            nn.Linear(n_hidden_units[0], n_hidden_units[1]),
            activation_functions[activation_fn],
            nn.Linear(n_hidden_units[1], n_output),
        )
    
    def forward(self, x):
        return self.model(x)

# %%
# 훈련 루프 함수
def training_loop(model, optimizer, train_data_loader, validation_data_loader, 
                  n_epochs, patience, verbose=True):
    loss_fn = nn.BCEWithLogitsLoss()
    next_print_epoch = 100
    
    # Early stopping 변수
    best_validation_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(1, n_epochs + 1):
        # Training
        loss_train = 0.0
        num_trains = 0
        model.train()
        
        for train_batch in train_data_loader:
            input_data = train_batch['input']
            target = train_batch['target'].float().unsqueeze(1)
            
            output_train = model(input_data)
            loss = loss_fn(output_train, target)
            loss_train += loss.item()
            num_trains += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Validation
        loss_validation = 0.0
        num_validations = 0
        model.eval()
        
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                input_data = validation_batch['input']
                target = validation_batch['target'].float().unsqueeze(1)
                
                output_validation = model(input_data)
                loss = loss_fn(output_validation, target)
                loss_validation += loss.item()
                num_validations += 1
        
        avg_train_loss = loss_train / num_trains
        avg_validation_loss = loss_validation / num_validations
        
        # Wandb 로깅
        wandb.log({
            "Epoch": epoch,
            "Training loss": avg_train_loss,
            "Validation loss": avg_validation_loss
        })
        
        # Early stopping 체크
        if avg_validation_loss < best_validation_loss:
            best_validation_loss = avg_validation_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        # 주기적 출력
        if verbose and epoch >= next_print_epoch:
            print(f"Epoch {epoch}, "
                  f"Training loss {avg_train_loss:.4f}, "
                  f"Validation loss {avg_validation_loss:.4f}, "
                  f"Patience: {patience_counter}/{patience}")
            next_print_epoch += 100
        
        # Early stopping 조건
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch}")
            print(f"Best validation loss: {best_validation_loss:.4f}")
            model.load_state_dict(best_model_state)
            break
    
    return model, best_validation_loss

# %%
# Test 예측 함수
def predict_test(model, test_data_loader):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for test_batch in test_data_loader:
            input_data = test_batch['input']
            output = model(input_data)
            predictions.append(output)
    
    predictions = torch.cat(predictions, dim=0).squeeze()
    return predictions

# %%
# CSV 저장 함수
def save_predictions_to_csv(predictions, filename="submission.csv"):
    CURRENT_FILE_PATH = os.getcwd()
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
    test_df = pd.read_csv(test_data_path)
    
    # 예측값을 0 또는 1로 변환
    predictions_binary = (torch.sigmoid(predictions) > 0.5).long().numpy()
    
    # 제출 파일 생성
    submission_df = pd.DataFrame({
        'PassengerId': test_df['PassengerId'],
        'Survived': predictions_binary
    })
    
    submission_df.to_csv(filename, index=False)
    print(f"Predictions saved to {filename}")
    return submission_df

c:\Users\didsu\miniconda3\envs\link_dl\lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
c:\Users\didsu\miniconda3\envs\link_dl\lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annot

BASE_PATH: c:\Users\didsu\workspace\deeplearning\link_dl\_04_your_code\homework_2


# 요구사항 1

_01_code/_08_learning_and_optimization/c_my_model_training_with_argparse_wandb.py 코드를 그대로 활용하되 titanic 데이터에 맞게 수정하여 코딩하기

–Wandb로 훈련 과정 데이터올려 그래프얻어내기
- Training loss
- Validation loss
- 위 두 그래프를 보여주는 WandbURL 얻어내기 : 

WandbURL 얻어내기 : https://wandb.ai/-ddj127-korea-university-of-technology-and-education/my_model_training?nw=nwuserddj127

In [2]:
config_req1 = {
    'epochs': 5000,
    'batch_size': 128,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
    'activation_fn': 'ReLU',
    'patience': 100,
}

wandb.init(
    mode="online",
    project="titanic_homework2",
    name="Req1_ReLU_b128_baseline",
    tags=["homework2", "requirement1", "baseline"],
    config=config_req1
)

print("=" * 60)
print("요구사항 1: 기본 훈련 시작")
print("=" * 60)

# 데이터 로드
train_loader, val_loader, test_loader = get_data(config_req1['batch_size'])

# 모델 및 옵티마이저 생성
model_req1 = MyModel(
    n_input=10, 
    n_output=1, 
    n_hidden_units=config_req1['n_hidden_unit_list'],
    activation_fn=config_req1['activation_fn']
)
optimizer_req1 = optim.SGD(model_req1.parameters(), lr=config_req1['learning_rate'])

# 훈련
model_req1, best_loss_req1 = training_loop(
    model_req1, optimizer_req1, train_loader, val_loader,
    config_req1['epochs'], config_req1['patience']
)

print(f"\n요구사항 1 완료! Best Validation Loss: {best_loss_req1:.4f}")
print(f"Wandb URL: {wandb.run.get_url()}")

wandb.finish()

wandb: Currently logged in as: -ddj127 (-ddj127-korea-university-of-technology-and-education) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


요구사항 1: 기본 훈련 시작


c:\Users\didsu\workspace\deeplearning\link_dl\_04_your_code\homework_2\titanic_dataset.py:129: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
c:\Users\didsu\workspace\deeplearning\link_dl\_04_your_code\homework_2\titanic_dataset.py:148: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.0      3      1  21.0750         2      0   
8       1.0       3    0  27.0      0      2  11.1333         2      3   
9       1.0       2    0  14.0      1      0  30.0708         0      3   

   family_num  alone  
0           1    0.

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


Wandb URL: https://wandb.ai/-ddj127-korea-university-of-technology-and-education/titanic_homework2/runs/q8qsbmjf


Epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇████
Training loss,█▄▃▃▃▃▃▃▃▃▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Validation loss,█▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▁▂▁▁▁
Epoch,1772
Training loss,0.54559
Validation loss,0.53467


# 요구사항 2

1. Activation Function 변경
- 목표: 모델 구성 내에서 활성화 함수(Activation Function)를 변경하여 더 나은 성능을 내는 함수를 찾습니다.
- 비교 대상: Sigmoid, ReLU, ELU, LeakyReLU

2. Batch Size 변경
- 목표: 모델 학습 시 배치 크기(Training Batch Size)를 변경하여 더 나은 성능을 내는 크기를 찾습니다.
- 비교 대상: 16, 32, 64, 128

args의 값들을 변경하면서 진행함

In [6]:
BEST_ACTIVATION = 'ReLU'  # 'Sigmoid', 'ReLU', 'ELU', 'Leaky ReLU' 중 선택
BEST_BATCH_SIZE = 128     # 16, 32, 64, 128 중 선택

print("=" * 60)
print("요구사항 2: 선택한 Activation Function & Batch Size로 훈련")
print("=" * 60)
print(f"선택한 조합:")
print(f"  - Activation Function: {BEST_ACTIVATION}")
print(f"  - Batch Size: {BEST_BATCH_SIZE}")

config_req2 = {
    'epochs': 5000,
    'batch_size': BEST_BATCH_SIZE,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
    'activation_fn': BEST_ACTIVATION,
    'patience': 100,
}

wandb.init(
    mode="online",
    project="titanic_homework2",
    name=f"Req2_{BEST_ACTIVATION}_b{BEST_BATCH_SIZE}",
    tags=["homework2", "requirement2", BEST_ACTIVATION, f"batch_{BEST_BATCH_SIZE}"],
    config=config_req2
)

# 데이터 로드
train_loader, val_loader, test_loader = get_data(BEST_BATCH_SIZE)

# 모델 및 옵티마이저 생성
model_req2 = MyModel(
    n_input=10, 
    n_output=1, 
    n_hidden_units=config_req2['n_hidden_unit_list'],
    activation_fn=BEST_ACTIVATION
)
optimizer_req2 = optim.SGD(model_req2.parameters(), lr=config_req2['learning_rate'])

# 훈련
model_req2, best_loss_req2 = training_loop(
    model_req2, optimizer_req2, train_loader, val_loader,
    config_req2['epochs'], config_req2['patience']
)

print(f"\n요구사항 2 완료! Best Validation Loss: {best_loss_req2:.4f}")
print(f"Wandb URL: {wandb.run.get_url()}")

wandb.finish()

Epoch 1800, Training loss 0.5524, Validation loss 0.5523, Patience: 1/100
Epoch 1900, Training loss 0.5462, Validation loss 0.5485, Patience: 1/100
Epoch 2000, Training loss 0.5415, Validation loss 0.5456, Patience: 6/100
Epoch 2100, Training loss 0.5312, Validation loss 0.5418, Patience: 0/100
Epoch 2200, Training loss 0.5390, Validation loss 0.5454, Patience: 3/100
Epoch 2300, Training loss 0.5320, Validation loss 0.5378, Patience: 1/100
Epoch 2400, Training loss 0.5224, Validation loss 0.5331, Patience: 9/100
Epoch 2500, Training loss 0.5209, Validation loss 0.5294, Patience: 1/100
Epoch 2600, Training loss 0.5160, Validation loss 0.5216, Patience: 4/100
Epoch 2700, Training loss 0.5094, Validation loss 0.5165, Patience: 0/100
Epoch 2800, Training loss 0.5010, Validation loss 0.5108, Patience: 1/100
Epoch 2900, Training loss 0.5009, Validation loss 0.5259, Patience: 5/100
Epoch 3000, Training loss 0.4921, Validation loss 0.5029, Patience: 4/100
Epoch 3100, Training loss 0.4894, Vali

Epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇█
Training loss,█▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▂▁▁▁▁▁▂▁
Validation loss,██▇▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▂▁▁▁▁▄▂▁▁
Epoch,5000
Training loss,0.42969
Validation loss,0.45434


# 요구사항 3

모델 구성: '요구사항 2'에서 찾은 가장 좋은 성능의 **활성화 함수(Activation Function)**와 **배치 크기(Batch Size)**를 사용하여 모델을 구성합니다.

데이터 준비: 사전에 테스트 데이터(test_data_loader)를 구성합니다.

테스트 시점 고찰: 훈련 과정 중 어느 에포크(Epoch) 시점에서 테스트를 수행해야 submission.csv 파일을 가장 효과적으로 생성할 수 있을지 고찰합니다.

추가 코딩: 위 고찰 내용을 바탕으로 필요한 코드를 추가로 구현합니다.

결과 제출: 최종적으로 submission.csv 파일을 생성합니다.

| 추가 코딩 구현했음
활성화 함수: ReLU
- 요구사항 2에서 다양한 활성화 함수(ReLU, Sigmoid, ELU, Leaky ReLU)를 비교 실험한 결과, ReLU가 가장 우수한 성능을 보임
- 선택 이유: 학습 속도가 빠르고 gradient vanishing 문제가 적으며, validation loss가 가장 낮게 수렴


배치 크기: 128
- 요구사항 2에서 여러 배치 크기(32, 64, 128, 256)를 실험한 결과, 128이 최적
- 선택 이유: 학습 안정성과 속도의 균형이 좋고, 메모리 효율적이면서도 gradient 업데이트가 안정적

csv 파일로 저장하는 코드는
- save_predictions_to_csv(...)로 진행함.
predict_test()에서 모든 배치의 예측값(logits)을 모은 다음, save_predictions_to_csv()에서 한 번에 sigmoid를 적용하고 이진화함

early stoping의 기준을 잡음.
```class Args:
    def __init__(self):
        ....
        self.patience = 100```

In [10]:
print("\n" + "=" * 60)
print("요구사항 3: Test 데이터 예측 및 Submission 파일 생성")
print("=" * 60)

# 요구사항 2에서 훈련한 model_req2를 그대로 사용
test_predictions = predict_test(model_req2, test_loader)
submission_df = save_predictions_to_csv(test_predictions, "submission.csv")

print(f"\n submission.csv 파일이 생성되었습니다!")
print(f"   사용된 모델: {BEST_ACTIVATION} + Batch Size {BEST_BATCH_SIZE}")
print(f"   Best Validation Loss: {best_loss_req2:.4f}")


요구사항 3: Test 데이터 예측 및 Submission 파일 생성
✅ Predictions saved to submission.csv

 submission.csv 파일이 생성되었습니다!
   사용된 모델: ReLU + Batch Size 128
   Best Validation Loss: 0.4395


# 요구사항 4

1. submission.csv 파일 제출

Kaggle에 로그인한 후, "Submit Prediction" 기능을 사용하여 submission.csv 파일을 제출합니다.

2. Leaderboard 순위 확인 및 캡처

Leaderboard에 등록된 자신의 점수와 순위가 나오도록 스크린 이미지를 캡처합니다.

3. Jupyter Notebook에 이미지 추가

캡처한 이미지를 클라우드에 업로드하여 URL을 생성합니다.

생성된 URL을 사용하여 이미지를 Jupyter Notebook에 넣습니다.

<img src="submission.png" width="1000">

<img src="submission_scoreAndRank.png" width="1000">

# 숙제 후기

1. 전체적인 진행방식
 - 16, 32, 64, 128 batch size와 epoch는 1000으로 loss의 변화 추이를 그래프로 파악. 이후 선별된 소수의 activation & batchsize를 통해 2000 epoch까지 진행. 최종 선별된 명명된  
 ReLU_b128을 통해서 5000과 10000까지 진행함.
 
 - ReLU_b128가 5xxx까지는 괜찮다가, 10000까지 진행된 상황을 보면, 그래프 선 추이가 많이 Flucation이 발생함을 확인함. 본인은 해당 문제를 과적합의 문제라고도 봄. 하여, 추가적인 조치가 필요하다는 결론을 내림.

 - 생각보다 activation과 batch만으로는 loss값의 하락이 쉽지가 않았음.

2. XGBBoost를 사용하기도 함.
 - model이라는 개념이 필요하지 않았다라는 점이 생소하기도 했고, 뭔가 다른 점들이 많아서 헷갈리는 부분도 많았음


## 학습 과정의 그래프 모음

16, 32, 64, 128 batch size와 epoch는 1000으로 다양한 activations들의 loss의 변화 추이를 그래프로 파악.

<img src="Multiples And All Activation Functions.png" width="1000">

하나의 예시로 ReLU_b64의 경우에 Validation loss의 경우에 위로 솟음이 보이는데 본인은 과적합의 현상으로 보여 최종 모델을 선별하기 위해서 이러한 추이가 보이는 것들은 제거함

<img src="위로는 올라가지만 아래로는 안내려가서 비교안하기.png" width="1000">

하단은 제거 결과

<img src="3개 선정 이후 다시 에포크 늘리기.png" width="1000">

이제 각각을 2000 에포크로 늘려봤음.

근데 ReLU batch 32은 과적합 신호로 인해서, Leaky ReLU batch 128의 경우엔 줄어드는 추이가 낮아서 제거함.

<img src="3개 선정 이후 1500epoch 결과.png" width="1000">

ReLU batch 128을 10000까지 에포크를 늘려봄. 

4K 에포크에서 과적합의 초기 추세가 보임.

4xxx epoch가 최종적으로 가장 적합한 모델의 상태라는 결론에 도달함.

<img src="ReLU_batch128을 epoch10000까지 한 결과.png" width="1000">

최종 결과( 5000 epoch로 다시 돌림)

<img src="ReLU_batch128 최종적으로 결과 모음.png" width="1000">